<a href="https://colab.research.google.com/github/anuragal/deep-learning/blob/master/S12/imagenet_resnet18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/anuragal/deep-learning
! ls 'deep-learning/tiny-imagenet-200/'

fatal: destination path 'deep-learning' already exists and is not an empty directory.
test  train  val  wnids.txt  words.txt


## Model

In [2]:
!pip install albumentations==0.4.5

import torch
import torch.nn as nn
import torch.nn.functional as F

!pip install torchsummary
from torchsummary import summary

from models.resnet18 import ResNet18

use_cuda = False #torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = ResNet18(num_classes=200).to(device)
# summary(model, input_size=(3, 32, 32))

cpu


## Run the Model

In [3]:
%matplotlib inline

In [4]:
import torch.nn as nn

from dataloader import ImageData
from models.resnet18 import ResNet18
from datamodel import DataModel
from optimizer import OptimizerFactory

from torch.optim.lr_scheduler import OneCycleLR

dataset_path = 'deep-learning/tiny-imagenet-200/'
# Loading data and transforming it
img_data = ImageData()

# Choose from "albumentations" or "pytorch". Default is "pytorch"
img_data.load_TINY_IMAGENET(dataset_path, "albumentations")

criterion = nn.CrossEntropyLoss()

# optimizer can be choose from "SGD" & "Adam"
optim_sgd = OptimizerFactory("SGD")
optimizer = optim_sgd.load(params=model.parameters(), lr=0.02, momentum=0.9, weight_decay=5e-4)

dm = DataModel(img_data, criterion, optimizer, None, num_of_epochs = 30, cal_misclassified = True)

  pip install torch-lr-finder -v --global-option="amp"


CUDA Available? False


### Find LR Max

In [5]:
lr = dm.find_lr_max(model, device)

IndexError: ignored

### LR Max

In [ ]:
lr = 0.015
print(lr)

## Run Model

In [ ]:
# training the dataset and then running test
scheduler = OneCycleLR(optimizer, max_lr=lr, epochs=2, steps_per_epoch = len(img_data.trainloader), 
                          anneal_strategy='cos',
                          base_momentum=0.85,
                          cycle_momentum=True,
                          div_factor=10,
                          final_div_factor=10000.0,
                          last_epoch=-1,
                          max_momentum=0.95,
                          pct_start=0.3
                        )
dm.schedular = scheduler
dm.optimizer = optim_sgd.load(params=model.parameters(), lr=lr/10, momentum=0.9, weight_decay=0.005)
dm.run_model(model, device)

## Plotting Test & Train Accuracy

In [ ]:
# dm.plot_test_train_accuracy()

## Plotting Validation Accuracy

In [ ]:
# dm.plot_loss_accuracy()

## Plotting Misclassified Images

In [ ]:
# dm.plot_misclassified()

In [ ]:
# dm.plot_GRADcam(["layer1","layer2","layer3"])